<a href="https://colab.research.google.com/github/graviraja/100-Days-of-NLP/blob/applications%2Fclassification/applications/classification/Document%20Classification%20with%20HAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!unzip "./drive/My Drive/glove.6B.zip"

Archive:  ./drive/My Drive/glove.6B.zip
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
  inflating: glove.6B.50d.txt        


In [3]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz
# !wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Clothing_Shoes_and_Jewelry_5.json.gz

--2020-06-07 13:25:33--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2460495 (2.3M) [application/x-gzip]
Saving to: ‘reviews_Musical_Instruments_5.json.gz’

reviews_Musical_Ins 100%[===================>]   2.35M   858KB/s    in 2.8s    

2020-06-07 13:25:36 (858 KB/s) - ‘reviews_Musical_Instruments_5.json.gz’ saved [2460495/2460495]



In [4]:
!ls

drive		   glove.6B.300d.txt			  sample_data
glove.6B.100d.txt  glove.6B.50d.txt
glove.6B.200d.txt  reviews_Musical_Instruments_5.json.gz


In [0]:
!gunzip reviews_Musical_Instruments_5.json.gz
# !gunzip reviews_Clothing_Shoes_and_Jewelry_5.json.gz

In [6]:
!ls

drive		   glove.6B.300d.txt		       sample_data
glove.6B.100d.txt  glove.6B.50d.txt
glove.6B.200d.txt  reviews_Musical_Instruments_5.json


In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
import os
import json
import time
import numpy as np
import pandas as pd

from nltk.tokenize import sent_tokenize
from fastai.text import Tokenizer, Vocab
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
datafile = 'reviews_Musical_Instruments_5.json'
# datafile = 'reviews_Clothing_Shoes_and_Jewelry_5.json'
glove_path = './glove.6B.50d.txt'

cuda


In [0]:
with open(datafile, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

texts = []
labels = []

for line in lines[:-1]:
    data = json.loads(line)
    texts.append(data["reviewText"].lower())
    labels.append(int(data["overall"]) - 1)

In [11]:
train_data, valid_data, train_label, valid_label = train_test_split(
        texts, labels, train_size=0.8, random_state=1
    )

print(f"Number of training data examples: {len(train_label)}")
print(f"Number of validation data examples: {len(valid_label)}")

Number of training data examples: 8208
Number of validation data examples: 2053


In [0]:
train_label = np.array(train_label, dtype="int32")
valid_label = np.array(valid_label, dtype="int32")

In [0]:
class HANPreprocessor:
    """
    Preprocessor to prepare the data for Hierarchical Attention Networks.
    It will tokenize a document into sentences and sentences into tokens
    """

    def __init__(self, max_vocab, min_freq, percentile, tokenizer):
        self.max_vocab = max_vocab
        self.min_freq = min_freq
        self.percentile = percentile
        self.tokenizer = tokenizer
        self.vocab = None

    def _make_sentences(self, texts):
        texts_sents = [sent_tokenize(text) for text in texts]
        print(f"Sample sentences: {texts_sents[0]}")
        return texts_sents
    
    def tokenize(self, texts):
        print(f"Processing {len(texts)} documents")
        texts_sents = self._make_sentences(texts)
        all_sents = [s for sent in texts_sents for s in sent]

        texts_length = [0] + [len(s) for s in texts_sents]
        range_idx = [sum(texts_length[: i + 1]) for i in range(len(texts_length))]

        print(f"Tokenizing {len(all_sents)} sentences")
        sents_tokens = self.tokenizer(all_sents)

        # calculating lengths of tokens in each sentence for padding purposes
        sents_length = [len(s) for s in sents_tokens]

        if self.vocab is None:
            self.vocab = Vocab.create(sents_tokens, max_vocab=self.max_vocab, min_freq=self.min_freq)

        sents_nums = [self.vocab.numericalize(s) for s in sents_tokens]

        # group sentences into documents
        texts_nums = [sents_nums[range_idx[i]: range_idx[i + 1]] for i in range(len(range_idx[:-1]))]

        # compute max lengths for padding purposes
        self.maxlen_sent = int(np.quantile(sents_length, q=self.percentile))
        self.maxlen_doc = int(np.quantile(texts_length[1:], q=self.percentile))

        print("Padding sentences and documents...")
        self.pad_token = self.vocab.stoi['xxpad']

        padded_texts = [pad_nested_sequences(r, self.maxlen_sent, self.maxlen_doc, self.pad_token) for r in texts_nums]
        return np.stack(padded_texts, axis=0)
    
    def transform(self, texts):
        return self.tokenize(texts)

In [0]:
def tokenizer(texts):
    tokens = Tokenizer().process_all(texts)
    print(f"sentence: {texts[0]}")
    print(f"tokens: {tokens[0]}")
    return tokens

In [0]:
def pad_sequences(seq, max_len, pad_idx):
    if len(seq) > max_len:
        return np.array(seq[:max_len]).astype("int32")
    else:
        res = np.zeros(max_len, dtype="int32") + pad_idx
        res[:len(seq)] = seq
        return res

In [0]:
def pad_nested_sequences(seq, maxlen_sent, maxlen_doc, pad_idx):
    if len(seq) == 0:
        return np.array([[pad_idx] * maxlen_sent] * maxlen_doc).astype("int32")

    # pad the sentences in all docs
    seq = [pad_sequences(s, maxlen_sent, pad_idx) for s in seq]

    # padding the documents
    if len(seq) > maxlen_doc:
        return np.array(seq[:maxlen_doc])
    else:
        res = np.array([[pad_idx] * maxlen_sent] * maxlen_doc).astype("int32")
        res[:len(seq)] = seq
        return res

In [0]:
def build_embeddings_matrix(vocab, glove_path, emb_dim=50):
    embeddings = {}
    with open(glove_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    
    print("creating word embeddings")

    # instead of zeros, initialize the embedding for missing words as mean value
    mean_word_vector = np.mean(list(embeddings.values()), axis=0)
    num_words = len(vocab.itos)
    embeddings_matrix = np.zeros((num_words, emb_dim))
    found_words = 0
    for i, word in enumerate(vocab.itos):
        embedding_vector = embeddings.get(word)

        if embedding_vector is not None:
            embeddings_matrix[i] = embedding_vector
            found_words += 1
        else:
            embeddings_matrix[i] = mean_word_vector
    
    print(f"Embeddings found for {(found_words / num_words) * 100} % of the words")
    return embeddings_matrix

In [0]:
MAX_VOCAB = 5000
MIN_FREQ = 5
PERCENTILE = 0.8
BATCH_SIZE = 32
processor = HANPreprocessor(MAX_VOCAB, MIN_FREQ, PERCENTILE, tokenizer)


In [19]:
train_seq = processor.transform(train_data)
valid_seq = processor.transform(valid_data)


Processing 8208 documents
Sample sentences: ['this mxl studio 24 usb microphone is a decent choice for those needing a portable or a good quality studio microphone.', 'we professionally produce instructional videos, both at our shop and at customer locations, and often need to voice over footage.', 'this mic will replace theblue microphones snowball usb microphone (textured white)we use in studio and also themxl mics mxl-studio 1 usb condenser microphone, cardioidwe had previously used for off-site recording.pros -* good voice quality; cleanly captures spoken voices* good physical quality; this is beefy mic that feels very well made* looks professional; while not important to the functionality, it makes a good impression on our customers* long usb cord; allows for placement away from pc* headphone jack; "real time monitor", handy to listen to live recording and playback* travels well; includes a protective travel case that fits all of componentscons -* no pop-filter included; normally 

In [20]:
emb_weights = build_embeddings_matrix(processor.vocab, glove_path)

creating word embeddings
Embeddings found for 96.74000000000001 % of the words


In [0]:
train_set = TensorDataset(
    torch.from_numpy(train_seq).long(),
    torch.from_numpy(train_label).long()
)

train_loader = DataLoader(
    dataset=train_set,
    batch_size=BATCH_SIZE,
    shuffle=True)


In [0]:
valid_set = TensorDataset(
    torch.from_numpy(valid_seq).long(),
    torch.from_numpy(valid_label).long()
)

valid_loader = DataLoader(
    dataset=valid_set,
    batch_size=BATCH_SIZE,
    shuffle=False)

In [0]:
def embedded_dropout(embed, words, dropout=0.1, scale=None):
    if dropout:
        mask = embed.weight.data.new().resize_((embed.weight.size(0), 1))
        mask = mask.bernoulli_(1 - dropout).expand_as(embed.weight) / (1 - dropout)
        masked_embed_weight = mask * embed.weight
    else:
        masked_embed_weight = embed.weight
    
    if scale:
        masked_embed_weight = scale.expand_as(masked_embed_weight) * masked_embed_weight
    padding_idx = embed.padding_idx

    emb = F.embedding(
        words,
        masked_embed_weight,
        padding_idx,
        embed.max_norm,
        embed.norm_type,
        embed.scale_grad_by_freq,
        embed.sparse
    )

    return emb

In [0]:
class LockedDropout(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x, dropout=0.5):
        if not dropout:
            return x
        mask = x.data.new(1, x.shape[1], x.shape[2])
        mask = mask.bernoulli_(1 - dropout) / (1 - dropout)
        mask = mask.expand_as(x)
        return mask * x

In [0]:
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()

        self.attn = nn.Linear(hidden_dim, hidden_dim)
        self.v = nn.Linear(hidden_dim, 1, bias=False)
    
    def forward(self, inp):
        # inp => [batch_size, seq_len, hidden_dim]

        energy = torch.tanh(self.attn(inp))
        # energy => [batch_size, seq_len, hidden_dim]

        attention = F.softmax(self.v(energy), dim=1)
        # attention => [batch_size, seq_len, 1]

        return attention

In [0]:

class WordAttention(nn.Module):
    def __init__(self, input_dim, emb_dim, emb_weights, pad_idx, hidden_dim):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx=pad_idx)
        self.embedding.weight = nn.Parameter(torch.tensor(emb_weights).float())
        self.rnn = nn.GRU(emb_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.word_attn = Attention(hidden_dim * 2)
        self.lockdrop = LockedDropout()

    def forward(self, inp, hidden, emb_dropout=0.1, lock_dropout=0.5):
        # inp => [batch_size, seq_len]
        # hidden => [num_dir * num_layers, batch_size, hidden_dim]

        # embedded = self.embedding(inp)
        embedded = embedded_dropout(self.embedding, inp, dropout=emb_dropout)
        # embedded => [batch_size, seq_len, emb_dim]

        if lock_dropout:
            embedded = self.lockdrop(embedded, lock_dropout)

        output, hidden = self.rnn(embedded, hidden)
        # output => [batch_size, seq_len, hidden_dim * num_dir]
        #        => [batch_size, seq_len, hidden_dim * 2]
        # hidden => [num_dir * n_layers, batch_size, hidden_dim]

        attention = self.word_attn(output)
        # attention => [batch_size, seq_len, 1]

        # output => [batch_size, seq_len, hid_dim * 2]
        weighted = torch.sum(attention * output, dim=1)
        # weighted => [batch_size, hidden_dim * 2]

        attention = attention.permute(0, 2, 1)
        # attention => [batch_size, 1, seq_len]

        weighted = weighted.unsqueeze(1)
        # weighted => [batch_size, 1, hidden_dim * 2]

        return attention, weighted, hidden

In [0]:
class SentenceAttention(nn.Module):
    def __init__(self, word_hidden_dim, sent_hidden_dim, pad_idx, dropout):
        super().__init__()

        self.rnn = nn.GRU(word_hidden_dim * 2, sent_hidden_dim, bidirectional=True)
        self.sent_attn = Attention(sent_hidden_dim * 2)
    
    def forward(self, inp):
        # inp => [batch_size, seq_len, word_hid_dim * 2]

        output, hidden = self.rnn(inp)
        # output => [batch_size, seq_len, sent_hid_dim * 2]
        # hidden => [num_layers * num_dir, batch_size, sent_hid_dim]

        attention = self.sent_attn(output)
        # attention => [batch_size, seq_len, 1]

        # output => [batch_size, seq_len, hid_dim * 2]
        weighted = torch.sum(attention * output, dim=1)
        # weighted => [batch_size, hidden_dim * 2]

        attention = attention.permute(0, 2, 1)
        # attention => [batch_size, 1, seq_len]

        return attention, weighted

In [0]:

class HierarchicalAttention(nn.Module):
    def __init__(self, input_dim, emb_dim, emb_weights, word_hid_dim, sent_hid_dim, pad_idx, output_dim, dropout, device):
        super().__init__()

        self.word_hid_dim = word_hid_dim
        self.device = device
        self.word_attention = WordAttention(input_dim, emb_dim, emb_weights, pad_idx, word_hid_dim)
        self.sent_attention = SentenceAttention(word_hid_dim, sent_hid_dim, pad_idx, dropout)

        self.fc = nn.Linear(sent_hid_dim * 2, output_dim)
    
    def forward(self, inp, emb_dropout=0.1, lock_dropout=0.5):
        # inp => [batch_size, max_sents, max_words]
        
        batch_size = inp.shape[0]
        inp = inp.permute(1, 0 , 2)
        # inp => [max_sents, batch_size, max_words]

        # initialize word rnn hiddens state
        hidden = torch.nn.Parameter(torch.zeros(2, batch_size, self.word_hid_dim)).to(self.device)

        word_attentions, sents_reps = [], []

        for sent in inp:
            word_attn, sent_rep, hidden = self.word_attention(sent, hidden, emb_dropout, lock_dropout)
            word_attentions.append(word_attn)
            sents_reps.append(sent_rep)
        
        sents = torch.cat(sents_reps, 1)
        # sents => [batch_size, max_sents, word_hid_dim * 2]

        word_attns = torch.cat(word_attentions, 1)
        # word_attns => [batch_size, max_sents, max_words]

        sent_attn, doc_rep = self.sent_attention(sents)
        sent_attn = sent_attn.squeeze(1)
        # sent_attn => [batch_size, max_sents]
        # doc_rep => [batch_size, sent_hid_dim * 2]

        logits = self.fc(doc_rep)
        # logits => [batch_size, 5]

        return logits, sent_attn, word_attns

In [0]:
input_dim = len(processor.vocab.itos)
emb_dim = 50
word_hid_dim = 32
sent_hid_dim = 32
pad_idx = processor.pad_token
output_dim = 5
dropout = 0.5
model = HierarchicalAttention(input_dim, emb_dim, emb_weights, word_hid_dim, sent_hid_dim, pad_idx, output_dim, dropout, device)
model = model.to(device)


In [30]:
def init_weights(model):
    for name, param in model.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

HierarchicalAttention(
  (word_attention): WordAttention(
    (embedding): Embedding(5000, 50, padding_idx=1)
    (rnn): GRU(50, 32, batch_first=True, bidirectional=True)
    (word_attn): Attention(
      (attn): Linear(in_features=64, out_features=64, bias=True)
      (v): Linear(in_features=64, out_features=1, bias=False)
    )
    (lockdrop): LockedDropout()
  )
  (sent_attention): SentenceAttention(
    (rnn): GRU(64, 32, bidirectional=True)
    (sent_attn): Attention(
      (attn): Linear(in_features=64, out_features=64, bias=True)
      (v): Linear(in_features=64, out_features=1, bias=False)
    )
  )
  (fc): Linear(in_features=64, out_features=5, bias=True)
)

In [0]:
optimizer = optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()


In [0]:
def train(model, iterator, criterion, optimizer, clip):
    model.train()
    epoch_loss = 0
    correct_count = 0
    total_count = 0
    for batch in iterator:
        optimizer.zero_grad()
        inp, labels = batch
        inp = inp.to(device)
        labels = labels.to(device)

        logits, _, _ = model(inp)
        # logits => [batch_size, 5]
        # labels => [batch_size]

        loss = criterion(logits, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
        
        top_k = logits.topk(1, 1)[1]
        true_k = labels.view(len(labels), 1).expand_as(top_k)
        correct_count += top_k.eq(true_k).float().sum().item()
        total_count += len(logits)
        
    
    accuracy = float(correct_count) / float(total_count)
    return epoch_loss / len(iterator), accuracy


In [0]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    correct_count = 0
    total_count = 0
    with torch.no_grad():
        for batch in iterator:
            inp, labels = batch
            inp = inp.to(device)
            labels = labels.to(device)

            # keep the emb_dropout, lock_dropout to 0 
            logits, _, _ = model(inp, 0, 0)
            # logits => [batch_size, 5]
            # labels => [batch_size]

            loss = criterion(logits, labels)
            epoch_loss += loss.item()

            top_k = logits.topk(1, 1)[1]
            true_k = labels.view(len(labels), 1).expand_as(top_k)
            correct_count += top_k.eq(true_k).float().sum().item()
            total_count += len(logits)
    
    accuracy = float(correct_count) / float(total_count)
    return epoch_loss / len(iterator), accuracy

In [0]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [35]:
N_EPOCHS = 10
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, CLIP)
    valid_loss, valid_acc = evaluate(model, valid_loader, criterion)
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 8s
	Train Loss: 0.937 | Train Acc: 67.57%
	 Val. Loss: 0.865 |  Val. Acc: 67.17%
Epoch: 02 | Epoch Time: 0m 8s
	Train Loss: 0.831 | Train Acc: 69.08%
	 Val. Loss: 0.873 |  Val. Acc: 67.36%
Epoch: 03 | Epoch Time: 0m 8s
	Train Loss: 0.767 | Train Acc: 70.39%
	 Val. Loss: 0.878 |  Val. Acc: 67.12%
Epoch: 04 | Epoch Time: 0m 8s
	Train Loss: 0.722 | Train Acc: 71.61%
	 Val. Loss: 0.910 |  Val. Acc: 63.08%
Epoch: 05 | Epoch Time: 0m 8s
	Train Loss: 0.684 | Train Acc: 73.31%
	 Val. Loss: 0.896 |  Val. Acc: 66.29%
Epoch: 06 | Epoch Time: 0m 8s
	Train Loss: 0.655 | Train Acc: 74.05%
	 Val. Loss: 0.952 |  Val. Acc: 65.81%
Epoch: 07 | Epoch Time: 0m 8s
	Train Loss: 0.623 | Train Acc: 75.02%
	 Val. Loss: 1.032 |  Val. Acc: 65.17%
Epoch: 08 | Epoch Time: 0m 8s
	Train Loss: 0.621 | Train Acc: 75.41%
	 Val. Loss: 1.012 |  Val. Acc: 66.49%
Epoch: 09 | Epoch Time: 0m 8s
	Train Loss: 0.608 | Train Acc: 76.05%
	 Val. Loss: 1.075 |  Val. Acc: 66.59%
Epoch: 10 | Epoch Time: 0m 8

In [36]:
model.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [0]:
import matplotlib
import IPython

In [0]:
def classify(document, processor, model, device):
    sentences = processor._make_sentences([document])
    inp = processor.transform([document])
    print(f"inp shape: {inp.shape}")
    inp = torch.LongTensor(inp).to(device)
    logits, sent_attn, word_attn = model(inp, 0, 0)
    return logits, sentences, sent_attn, word_attn

In [133]:
idx = 143
valid_data[idx], valid_label[idx]

("this is a fun pick.  i play a lot with my fingers and thumb and thought i'd give this a try.  its great!  i love the tone of this pick striking strings.  i love the way it transmits the feel of the strings.  more organic than plastic picks.  yes it might crumble on you if you play too hard.  i think if you play as hard with this pick as you would with your thumb you'll be fine.  its a lot of fun and for such a little price can open your playing to different tones for your instrument.  worth a try.",
 4)

In [134]:
logits, sentences, sent_attn, word_attn = classify(valid_data[idx], processor, model, device)

Sample sentences: ['this is a fun pick.', "i play a lot with my fingers and thumb and thought i'd give this a try.", 'its great!', 'i love the tone of this pick striking strings.', 'i love the way it transmits the feel of the strings.', 'more organic than plastic picks.', 'yes it might crumble on you if you play too hard.', "i think if you play as hard with this pick as you would with your thumb you'll be fine.", 'its a lot of fun and for such a little price can open your playing to different tones for your instrument.', 'worth a try.']
Processing 1 documents
Sample sentences: ['this is a fun pick.', "i play a lot with my fingers and thumb and thought i'd give this a try.", 'its great!', 'i love the tone of this pick striking strings.', 'i love the way it transmits the feel of the strings.', 'more organic than plastic picks.', 'yes it might crumble on you if you play too hard.', "i think if you play as hard with this pick as you would with your thumb you'll be fine.", 'its a lot of fun

In [135]:
print(logits.shape)
logits

torch.Size([1, 5])


tensor([[-2.3802, -2.7388, -0.9391,  0.5919,  2.0912]], device='cuda:0',
       grad_fn=<AddmmBackward>)

In [136]:
print(sent_attn.shape)
sent_attn

torch.Size([1, 10])


tensor([[0.0969, 0.1094, 0.1054, 0.1086, 0.1159, 0.1075, 0.0730, 0.0808, 0.1096,
         0.0929]], device='cuda:0', grad_fn=<SqueezeBackward1>)

In [0]:
def plot_sent_attn(sents, sents_attns, cmap="Greens"):
    cmap = matplotlib.cm.get_cmap(cmap)
    template = '<font face="monospace" \nsize="3"; span class="barcode"; style="color: black; background-color: {}">{}</span>'
    colored_doc = ""
    sents_attns = sents_attns.squeeze(0).cpu().detach().numpy()
    for sent, sent_w in zip(sents, sents_attns):
        sent = ' '.join([t for t in sent.split() if t!='xxpad'])
        if len(sent) > 0:
            color = matplotlib.colors.rgb2hex(cmap(sent_w)[:3])
            colored_doc += template.format(color, "&nbsp" + sent + "&nbsp") + "</br>"
    return colored_doc

In [0]:
sent_attn_html = plot_sent_attn(sentences[0], sent_attn)

In [139]:
IPython.display.HTML(sent_attn_html)

In [140]:
word_attn.shape

torch.Size([1, 10, 18])

In [0]:
def plot_word_attn(sentences, word_attn, cmap):
    word_attn = word_attn.squeeze(0).cpu().detach().numpy()
    cmap = matplotlib.cm.get_cmap(cmap)
    template = '<font face="monospace" \nsize="3"; span class="barcode"; style="color: black; background-color: {}">{}</span>'
    colored_doc = ""
    for sent, sent_w in zip(sentences, word_attn):
        sent_len, pad_count = len(sent.split()), 0
        for t, w in zip(sent.split(), sent_w):
            if t == 'xxpad':
                pad_count+=1
                continue
            color = matplotlib.colors.rgb2hex(cmap(w)[:3])
            colored_doc += template.format(color, "&nbsp" + t + "&nbsp")
        if pad_count != sent_len:
            colored_doc += "</br>"
    return colored_doc

In [0]:
word_attn_html = plot_word_attn(sentences[0], word_attn, 'Greens')

In [143]:
IPython.display.HTML(word_attn_html)